# Chapter 25 E Pluribus Monad

In [1]:
newtype Compose f g a = Compose { getCompose :: f (g a) } deriving (Eq, Show)

instance (Functor f, Functor g) => Functor (Compose f g) where
    fmap f (Compose fga) = Compose $ (fmap . fmap) f fga
    
{-# LANGUAGE InstanceSigs #-}
instance (Applicative f, Applicative g) => Applicative (Compose f g) where
    pure :: a -> Compose f g a
    pure = Compose . pure . pure
    (<*>) :: Compose f g (a -> b)
             -> Compose f g a
             -> Compose f g b
    (Compose fg_f) <*> (Compose fg_a) =
        Compose $ (<*>) <$> fg_f <*> fg_a -- first <*> is for 'g', <$> is also for 'g' and last <*> is for 'f'
        
instance (Foldable f, Foldable g) => Foldable (Compose f g) where
    foldMap :: Monoid m => (a -> m) -> Compose f g a -> m
    foldMap f (Compose fga) = foldMap (foldMap f) fga 

instance (Traversable f, Traversable g) => Traversable (Compose f g) where
    traverse :: Applicative h => (a -> h b) -> Compose f g a -> h (Compose f g b)
    traverse f (Compose fga) = Compose <$> traverse (traverse f) fga

In [2]:
class Bifunctor p where
    {-# MINIMAL bimap | first, second #-}
    bimap :: (a -> b)
             -> (c -> d)
             -> p a c
             -> p b d
    bimap f g = first f . second g
    
    first :: (a -> b) -> p a c -> p b c
    first f = bimap f id
    
    second :: (b -> c) -> p a b -> p a c
    second = bimap id
    
data Deux a b = Deux a b

instance Bifunctor Deux where
    bimap f g (Deux a b) = Deux (f a) (g b)
    
data Const a b = Const a

instance Bifunctor Const where
    bimap f _ (Const a) = Const (f a) 
    
data Drei a b c = Drei a b c

instance Bifunctor (Drei x) where
    bimap f g (Drei x a b) = Drei x (f a) (g b)
    
instance Bifunctor Either where
    bimap f g e = case e of
        Left  l -> Left  (f l)
        Right r -> Right (g r)